In [3]:
#!/usr/bin/env python                                                                                                                                                 
# Import libraries                                                                                                                                                    
import sys
import argparse
import time
import ast
import re

class args():
    pass
args.model_name = "allenai/scibert_scivocab_uncased"
args.model_name = "distilbert-base-uncased"
args.model_name = "climatebert__distilroberta-base-climate-f"
#args.model_name = "svm"
args.y_prefix = "4 -"
args.n_splits = 3
args.make_predictions = "False"

In [2]:
#!/usr/bin/env python
# Import libraries
import sys
import argparse
import time
import ast

class args():
    pass
args.model_name = "allenai/scibert_scivocab_uncased"
args.model_name = "distilbert-base-uncased"
args.model_name = "climatebert__distilroberta-base-climate-f"
#args.model_name = "svm"
args.y_prefix = "4 -"
args.n_splits = 3
args.make_predictions = "False"

t0 = time.time()

# Establish what task number we have if running from slurm, otherwise just get a random number
# This means we are just running the script in test mode
try:
    from mpi4py import MPI
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    test = False
except:
    import random
    rank = random.randint(0,args.n_splits**2)
    print(rank)
    test = True
    
rank_i = rank%args.n_splits
rank_j = rank//args.n_splits

print("Rank I ", rank_i, "Rank j", rank_j)
# Import the rest of the libraries
import gc
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import GridSearchCV, cross_val_score, cross_validate, KFold
import pickle
import re

# Load data
seen_df = pd.read_csv('../data/0_labelled_documents.csv')

seen_df = (seen_df
      .sort_values('id')
      .sample(frac=1, random_state=1)
      .reset_index(drop=True)
)

weights_df = pd.read_csv('../data/0_label_weights.csv')

# Get the target labels from the y_prefix argument passed to this script
if len(args.y_prefix) < 2:
    args.y_prefix+=" "
cols = [x for x in seen_df.columns if re.match(f"^{args.y_prefix}",x)]
print(cols)
num_labels=len(cols)

# If the target is inclusion, use only those documents for which we have a non-na value
# Otherwise, only use those documents which are included
# also define what subset is to be treated as a random representative sample
# For labels beyond inclusion, we treat all those that are representative of the included
# studies as representative
if "INCLUDE" in args.y_prefix:
    y_var = cols[0]
    seen_df = seen_df.loc[pd.notna(seen_df[y_var]),:].reset_index(drop=True)
    seen_df['random'] = seen_df['representative_sample']
else:
    seen_df = seen_df[seen_df['INCLUDE']==1]
    seen_df['random'] = seen_df['representative_relevant']
    

# Turn the columns into target variables and get class-weights to counteract class imbalances
if len(cols)==1:
    y_var = cols[0]
    seen_df = seen_df.loc[pd.notna(seen_df[y_var]),:].reset_index(drop=True)
    print(seen_df.shape)
    seen_df['labels'] = list(seen_df[y_var].values.astype(int))
    cw = seen_df[(seen_df['random']==1) & (seen_df[y_var]==0)].shape[0] / seen_df[(seen_df['random']==1) & (seen_df[y_var]==1)].shape[0]
    class_weight={1:cw}
    scorer = "F1"
    weights_df["sample_weight"] = list(weights_df[y_var+"_sample_weight"].fillna(1).values)
else:
    num_labels = len(cols) 
    weights_df['sample_weight'] = list(weights_df[[x+"_sample_weight" for x in cols]].fillna(1).values)
    seen_df = seen_df.replace(2,1)
    seen_df['labels'] = list(seen_df[cols].values.astype(int))
    seen_df = seen_df.dropna(subset=cols)
    seen_df = seen_df.reset_index(drop=True)
    scorer = "F1 macro"
    class_weight = {}
    for i, t in enumerate(cols):
        cw = seen_df[(seen_df['random']==1) & (seen_df[t]==0)].shape[0] / seen_df[(seen_df['random']==1) & (seen_df[t]==1)].shape[0]
        class_weight[i] = cw

# Remove unneccessary columns
seen_df = seen_df[["id","title","content","labels","random"]+cols].merge(
    weights_df[["doc__id","sample_weight"]].rename(columns={"doc__id":"id"})
)


# Merge with the unseen data if necessary
seen_df['seen']  = 1
if args.make_predictions=="True":
    unseen_df = pd.read_csv('../data/0_unlabelled_documents.csv')
    unseen_df['seen'] = 0
    df = (pd.concat([seen_df,unseen_df])
          .sort_values('id')
          .sample(frac=1, random_state=1)
          .reset_index(drop=True)
    )
    df.content = df.content.astype(str)
else:
    df = seen_df

# This is the index of nonrandom/nonrepresentative documents, and these will be removed from validation sets
nonrandom_index = df[(df['random']!=1) & (df['seen']==1)].index
random_index = df[df['random']==1].index
seen_index = df[df['seen']==1].index
unseen_index = df[df['seen']==0].index

print("seen_index", seen_index)
print("nonrandom_index", nonrandom_index)

# Try this with tensorflow if this is True, otherwise we do it with pytorch
try_tf = False
if "distilbert" in args.model_name.lower() and try_tf:
    # Start setting up the Deep learning things
    from transformers import BertTokenizer, DistilBertTokenizer,  TFBertForSequenceClassification, TFDistilBertForSequenceClassification
    import tensorflow as tf
    gpus = tf.config.experimental.list_physical_devices('GPU')
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)

    tf.keras.mixed_precision.experimental.set_policy('mixed_float16')
    #tf.config.threading.set_intra_op_parallelism_threads(1)
    #tf.config.threading.set_inter_op_parallelism_threads(1)

    import tensorflow_addons as tfa
    use_tf = True
else:
    import torch
    use_tf = False

# import the helper functions defined in cv_setup.py
import cv_setup as cvs

# Get the BERT parameters, and include class_weight as a parameter to be tested
bert_params = cvs.bert_params
bert_params['class_weight'].append(class_weight)
bert_params['class_weight'] = [class_weight]
param_space = list(cvs.product_dict(**bert_params))
params = list(bert_params.keys())

if test:
    param_space = param_space[:2]

# Get the spilts for our outer fold, discard=False means to pass the nonrandom documents that
# would ordinarily be in the validation set back into the test set
outer_cv = cvs.KFoldRandom(args.n_splits, seen_index, nonrandom_index, discard=False)

# Iterate through the folds
for k, (train, test) in enumerate(outer_cv):    
    print(len(train), len(test))

9
Rank I  0 Rank j 3
['4 - 1. Economic instruments', '4 - 2. Regulatory Instruments', '4 - 3. Information, education and training', '4 - 4. Governance, strategies and targets', '4 - 5. Agreements']
seen_index Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            793, 794, 795, 796, 797, 798, 799, 800, 801, 802],
           dtype='int64', length=803)
nonrandom_index Int64Index([  7,  19,  37,  45,  50,  64, 154, 160, 217, 221, 227, 231, 234,
            237, 243, 253, 294, 336, 343, 359, 376, 389, 446, 487, 505, 513,
            547, 594, 595, 596, 599, 618, 627, 672, 705, 707, 713, 731, 733,
            757, 765, 778],
           dtype='int64')
551 252
545 258
552 251


In [5]:
# Get the target labels from the y_prefix argument passed to this script                                                                                              
if len(args.y_prefix) < 2:
    args.y_prefix+=" "
cols = [x for x in seen_df.columns if re.match(f"^{args.y_prefix}",x)]
print(cols)
num_labels=len(cols)

# If the target is inclusion, use only those documents for which we have a non-na value                                                                               
# Otherwise, only use those documents which are included                                                                                                              
# also define what subset is to be treated as a random representative sample                                                                                          
# For labels beyond inclusion, we treat all those that are representative of the included                                                                             
# studies as representative                                                                                                                                           
if "INCLUDE" in args.y_prefix:
    y_var = cols[0]
    seen_df = seen_df.loc[pd.notna(seen_df[y_var]),:].reset_index(drop=True)
    seen_df['random'] = seen_df['representative_sample']
else:
    seen_df = seen_df[seen_df['INCLUDE']==1]
    seen_df['random'] = seen_df['representative_relevant']


# Turn the columns into target variables and get class-weights to counteract class imbalances                                                                         
if len(cols)==1:
    y_var = cols[0]
    seen_df = seen_df.loc[pd.notna(seen_df[y_var]),:].reset_index(drop=True)
    print(seen_df.shape)
    seen_df['labels'] = list(seen_df[y_var].values.astype(int))
    cw = seen_df[(seen_df['random']==1) & (seen_df[y_var]==0)].shape[0] / seen_df[(seen_df['random']==1) & (seen_df[y_var]==1)].shape[0]
    class_weight={1:cw}
    scorer = "F1"
    weights_df["sample_weight"] = list(weights_df[y_var+"_sample_weight"].fillna(1).values)
else:
    num_labels = len(cols)
    weights_df['sample_weight'] = list(weights_df[[x+"_sample_weight" for x in cols]].fillna(1).values)
    seen_df = seen_df.replace(2,1)
    seen_df['labels'] = list(seen_df[cols].values.astype(int))
    seen_df = seen_df.dropna(subset=cols)
    seen_df = seen_df.reset_index(drop=True)
    scorer = "F1 macro"
    class_weight = {}
    for i, t in enumerate(cols):
        cw = seen_df[(seen_df['random']==1) & (seen_df[t]==0)].shape[0] / seen_df[(seen_df['random']==1) & (seen_df[t]==1)].shape[0]
        class_weight[i] = cw

# Remove unneccessary columns                                                                                                                                         
seen_df = seen_df[["id","title","content","labels","random"]+cols].merge(
    weights_df[["doc__id","sample_weight"]].rename(columns={"doc__id":"id"})
)


['4 - 1. Economic instruments', '4 - 2. Regulatory Instruments', '4 - 3. Information, education and training', '4 - 4. Governance, strategies and targets', '4 - 5. Agreements']


In [6]:

# Merge with the unseen data if necessary                                                                                                                             
seen_df['seen']  = 1
if args.make_predictions=="True":
    unseen_df = pd.read_csv('../data/0_unlabelled_documents.csv')
    unseen_df['seen'] = 0
    df = (pd.concat([seen_df,unseen_df])
          .sort_values('id')
          .sample(frac=1, random_state=1)
          .reset_index(drop=True)
    )
    df.content = df.content.astype(str)
else:
    df = seen_df


In [30]:
# This is the index of nonrandom/nonrepresentative documents, and these will be removed from validation sets                                                          
nonrandom_index = df[(df['random']!=1) & (df['seen']==1)].index
random_index = df[df['random']==1].index
seen_index = df[df['seen']==1].index
unseen_index = df[df['seen']==0].index

# Try this with tensorflow if this is True, otherwise we do it with pytorch                                                                                           
try_tf = False
if "distilbert" in args.model_name.lower() and try_tf:
    # Start setting up the Deep learning things                                                                                                                       
    from transformers import BertTokenizer, DistilBertTokenizer,  TFBertForSequenceClassification, TFDistilBertForSequenceClassification
    import tensorflow as tf
    gpus = tf.config.experimental.list_physical_devices('GPU')
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)

    tf.keras.mixed_precision.experimental.set_policy('mixed_float16')
    #tf.config.threading.set_intra_op_parallelism_threads(1)                                                                                                          
    #tf.config.threading.set_inter_op_parallelism_threads(1)                                                                                                          

    import tensorflow_addons as tfa
    use_tf = True
else:
    import torch
    use_tf = False

# import the helper functions defined in cv_setup.py                                                                                                                  
import cv_setup as cvs

# Get the BERT parameters, and include class_weight as a parameter to be tested                                                                                       
bert_params = cvs.bert_params
bert_params['class_weight'].append(class_weight)
bert_params['class_weight'] = [class_weight]
param_space = list(cvs.product_dict(**bert_params))
params = list(bert_params.keys())

def KFoldRandom(n_splits, X, no_test, shuffle=False, discard=True):
    kf = KFold(n_splits=n_splits, shuffle=shuffle)
    for train, test in kf.split(X):
        X = np.array(X)
        train = X[train]
        test = X[test]
        if not discard:
            train = list(train) +  [x for x in test if x in no_test]
        test = [x for x in test if x not in no_test]
        yield (train, test)


outer_cv = KFoldRandom(args.n_splits, seen_index, nonrandom_index, discard=False)

# Iterate through the folds                                                                                                                                           
for k, (train, test) in enumerate(outer_cv):
    print(k)
    print(len(test))

0
253
1
258
2
249


In [36]:
outer_cv = KFoldRandom(args.n_splits, seen_index, nonrandom_index, discard=False)

total = 0
# Iterate through the folds                                                                                                                                           
for k, (train, test) in enumerate(outer_cv):
    print(len(test))
    total+=len(test)
    print(len(set(train) & set(test)))
    
total

253
0
258
0
249
0


760